# Output statistics

After running your MIKE simulation, you would often want to make different kinds of summary statistics of your data - both for your own understanding and for communicating your results.

Examples of statistics

* Min, max, mean, standard deviation 
* Quantiles/percentiles (e.g. median, interquartile range, return period etc)
* Probability/frequency of exceedance

Types of aggregations

* Total - aggregate all data to a single number 
* Temporal - aggregate all time steps; for a dfsu 2d, the result would be a map
* Spatial - aggregate all elements to a single value per time step
* Others: monthly, by layer, spatial bin, sub domain etc... 

Ways of calculating

* [MIKEIO.Dataset](https://dhi.github.io/mikeio/dataset.html#methods) 
* [MIKEIO.generic](https://dhi.github.io/mikeio/generic.html) (temporal aggregations only; larger-than-memory)
* custom code (typically with NumPy)


:::{note}
This section uses features introduced in MIKE IO v 0.10.1 (released 2021-10-26)
:::

In [ ]:
from mikeio import Dfsu, generic

## Dataset

For smaller dfs files (maybe up to 2GB) it can be convenient to read the data to memory before doing aggregations. The MIKEIO.Dataset class have several [methods](https://dhi.github.io/mikeio/dataset.html#methods) for aggregating data along an axis. See the generic section below for larger-than-memory data. 

In [ ]:
dfs = Dfsu("data/NorthSea_HD_and_windspeed.dfsu")
ds = dfs.read()
ds

### Temporal aggregations: mean

The default is to aggregate along the time axis - the output will therefore be a map.

In [ ]:
dsm  = ds.mean()
mean_ws = dsm["Wind speed"]
mean_ws.shape

In [ ]:
dfs.plot(z=mean_ws)

### Spatial aggregations 

The Dataset aggregation methods (e.g. mean) takes an `axis` argument. If we give it the spatial axis (or the string 'space'), it will produce a time series of spatially aggregated values. 


:::{note}
It's important to note that the spatial aggregations here ignores element areas! Only average takes a weights argument. 
:::

In [ ]:
df = ds.mean(axis="space").to_dataframe()
df.head()

In [ ]:
df['Wind speed'].plot();

Dataset has other methods for calculating typical statistics, e.g. max, quantile...

In [ ]:

ds[["Wind speed"]].max(axis="space").to_dataframe().plot(title="Max wind speed");

In [ ]:
ds[["Wind speed"]].quantile(q=[0.1,0.5,0.9],axis="space").to_dataframe().plot();

It's important to know that the element area is not taking into account when doing the spatial aggregations! Only Dataset.average supports weighted averages.

In [ ]:
df = ds[["Wind speed"]].mean(axis="space").to_dataframe()
df.columns = ["Simple mean"]
area=dfs.get_element_area()
df['Weighted'] = ds[["Wind speed"]].average(axis="space", weights=area).to_dataframe()
df.plot(title="Mean wind speed (simple vs weighted by element area)");

### Quantiles to file

In [ ]:
dsq  = ds.quantile(q=[0.1,0.5,0.9])
dsq

Write to a new dfsu file

In [ ]:
dfs.write("NorthSea_Quantiles.dfsu", dsq)

![](images/mikezero_quantiles.png)

### Total

Aggregating over all data (both time and space) can be done from the Dataset in a few ways: 

* ds.describe() - will give you summary statistics like pandas df.describe()
* using axis=None in ds.mean(), ds.min()
* using standard NumPy aggregation functions on the Dataset data e.g. ds["Wind speed"].mean()

In [ ]:
ds.describe()

In [ ]:
ds.min(axis=None).to_dataframe()

In [ ]:
ds["Wind speed"].min()

## Generic

The [MIKEIO.generic](https://dhi.github.io/mikeio/generic.html) submodule can produce common **temporal** statistics on any dfs file (of any size). The output will be a new dfs file. Currently, generic has these methods for calculating statistics: 

* avg_time()
* quantile()

In [ ]:
generic.avg_time("data/NorthSea_HD_and_windspeed.dfsu", "NorthSea_avg.dfsu")

In [ ]:
dfs = Dfsu("NorthSea_avg.dfsu")
ds = dfs.read(items="Wind speed")
ds

In [ ]:
dfs.plot(ds["Wind speed"], title="Mean wind speed"); # this will fail for MIKE IO < 0.9.1 

In [ ]:
generic.quantile("data/NorthSea_HD_and_windspeed.dfsu", "NorthSea_Quantiles2.dfsu", q=[0.1, 0.5, 0.9])

![](images/mikezero_quantiles.png)

## Custom



In [ ]:
dfs = Dfsu("data/NorthSea_HD_and_windspeed.dfsu")
ds = dfs.read()

### Dataset.aggregate

With aggregate we can get Dataset statistics with our "own" function, e.g. standard deviation:

In [ ]:
import numpy as np
dsa = ds.aggregate(func=np.std)

In [ ]:
dfs.plot(dsa["Wind speed"], label="Std [m/s]");

### Exceedance probability

Let's find out how often the wind exceeds 12m/s in our simulation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mikeio import Dataset, eum

In [ ]:
nt = ds.n_timesteps
one_to_zero = 1. - np.arange(1., nt + 1.)/nt

In [ ]:
val = ds["Wind speed"][:,0]
plt.plot(ds.time, val);
plt.plot(ds.time[val>12], val[val>12],'.r');
plt.axhline(y=12,color='r')
plt.ylabel('Wind speed [m/s]')
plt.title('How often is the wind speed above 12m/s (element 0)?');


In [ ]:
plt.plot(np.sort(ds["Wind speed"][:,0]), one_to_zero);
plt.xlabel('Wind speed [m/s]')
plt.ylabel('Probability of exceedance [-]')
plt.axvline(x=12,color='r')
plt.title('Wind speed exceedance in element 0');

In [ ]:
# Create empty Dataset 
items=[eum.ItemInfo(eum.EUMType.Probability)]
dse = Dataset(data=dfs.n_elements, time="2017-10-27", items=items)

In [ ]:
threshold = 12
for j in range(ds.n_elements):
    # this is a naive and slow way of calculating this!
    dat = ds["Wind speed"][:,j]
    dse[0][0,j] = np.interp(threshold, np.sort(dat), one_to_zero)

In [ ]:
dfs.plot(dse*100, title="Wind speed exceeding 12 m/s", 
    label="Time of Exceedance [%]", cmap="YlOrRd");

In [ ]:
total_hours = (ds.time[-1]-ds.time[0]).total_seconds()/3600
dfs.plot(dse*total_hours, title="Wind speed exceeding 12 m/s", 
    label="Exceedance [Hours]", cmap="YlOrRd");